In [4]:
import pandas as pd


pd.set_option('display.max_rows',500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Data Preparation

* Focus is always to undersand the final data structure
* Support each step by visual analytics

## John Hopkins GITHUB csv data

Error : b'From https://github.com/CSSEGISandData/COVID-19\n   21358a26..18b8ce4d  web-data   -> origin/web-data\n'
out : b'Already up to date.\n'


In [5]:
data_path = '../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw = pd.read_csv(data_path)

In [6]:
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38165,38196,38243,38288,38304,38324,38398,38494,38520,38544
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,9513,9606,9728,9844,9967,10102,10255,10406,10553,10704
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,44494,44833,45158,45469,45773,46071,46364,46653,46938,47216
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,1176,1184,1199,1199,1215,1215,1215,1261,1261,1301
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,2654,2729,2777,2805,2876,2935,2965,2981,3033,3092


In [4]:
page = requests.get('https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html')

In [5]:
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
html_table = soup.find('table')

In [7]:
all_rows = html_table.find_all('tr')

In [8]:
final_data_list = []

In [9]:
for pos, rows in enumerate(all_rows):
    
    col_list = [each_col.get_text(strip=True) for each_col in rows.find_all('td')]
    final_data_list.append(col_list)
    

In [10]:
pd.DataFrame(final_data_list).dropna().rename(columns={0:'State',1:'Cases',2:'Change',3:'Cases_in_last_7_days',4:'7_days_incidence',5:'deaths'})

,State,Cases,Change,Cases_in_last_7_days,7_days_incidence,deaths
2,Baden-Württem­berg,44.396,+278,1.500,"13,6",1.868
3,Bayern,60.759,+638,2.396,"18,3",2.644
4,Berlin,11.951,+72,459,"12,2",226
5,Branden­burg,3.966,+7,64,"2,5",169
6,Bremen,2.094,+9,58,"8,5",58
7,Hamburg,6.626,+99,213,"11,6",267
8,Hessen,16.500,+125,698,"11,1",537
9,Meck­lenburg-Vor­pommern,1.044,+6,29,"1,8",20
10,Nieder­sachsen,17.572,+118,495,"6,2",665
11,Nord­rhein-West­falen,61.201,+323,1.589,"8,9",1.827


## REST API calls

In [11]:
data = requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [12]:
json_object = json.loads(data.content)

In [13]:
type(json_object)

dict

In [14]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'serverGens', 'geometryType', 'spatialReference', 'fields', 'features'])

In [15]:
full_list = []

for pos, each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [16]:
full_list

[{'OBJECTID_1': 1,
  'LAN_ew_AGS': '01',
  'LAN_ew_GEN': 'Schleswig-Holstein',
  'LAN_ew_BEZ': 'Land',
  'LAN_ew_EWZ': 2896712,
  'OBJECTID': 15,
  'Fallzahl': 4197,
  'Aktualisierung': 1599688800000,
  'AGS_TXT': '01',
  'GlobalID': 'fc5ba936-c95c-432c-8a33-9eb2f30b660f',
  'faelle_100000_EW': 144.888411412664,
  'Shape__Area': 45737314116.8701,
  'Shape__Length': 2881496.1295035,
  'Death': 161},
 {'OBJECTID_1': 2,
  'LAN_ew_AGS': '02',
  'LAN_ew_GEN': 'Hamburg',
  'LAN_ew_BEZ': 'Freie und Hansestadt',
  'LAN_ew_EWZ': 1841179,
  'OBJECTID': 6,
  'Fallzahl': 6626,
  'Aktualisierung': 1599688800000,
  'AGS_TXT': '02',
  'GlobalID': '0f3e860c-5181-4d3f-a421-1d51f50315ea',
  'faelle_100000_EW': 359.878099847978,
  'Shape__Area': 2089396054.82129,
  'Shape__Length': 418800.187474072,
  'Death': 267},
 {'OBJECTID_1': 3,
  'LAN_ew_AGS': '03',
  'LAN_ew_GEN': 'Niedersachsen',
  'LAN_ew_BEZ': 'Land',
  'LAN_ew_EWZ': 7982448,
  'OBJECTID': 9,
  'Fallzahl': 17572,
  'Aktualisierung': 1599688800

In [17]:
pd.DataFrame(full_list)

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,4197,1599688800000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,144.888411,4.573731e+10,2.881496e+06,161
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,6626,1599688800000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,359.878100,2.089396e+09,4.188002e+05,267
2,3,03,Niedersachsen,Land,7982448,9,17572,1599688800000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,220.132972,1.299836e+11,4.008988e+06,665
3,4,04,Bremen,Freie Hansestadt,682986,5,2094,1599688800000,04,4132268b-54de-4327-ac1e-760e915112f1,306.594864,1.119157e+09,3.357177e+05,58
4,5,05,Nordrhein-Westfalen,Land,17932651,10,61201,1599688800000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,341.282502,8.782936e+10,2.648673e+06,1827
5,6,06,Hessen,Land,6265809,7,16500,1599688800000,06,93277ac4-e8fc-48c7-8940-028dc2ed66af,263.333913,5.235913e+10,2.148244e+06,537
6,7,07,Rheinland-Pfalz,Land,4084844,11,9560,1599688800000,07,e9b4296f-9be2-4e53-9a58-ccf1396cb03d,234.035865,4.783877e+10,1.774430e+06,245
7,8,08,Baden-Württemberg,Land,11069533,1,44396,1599688800000,08,80394ddf-c6a4-4a6e-be8e-0259a81b22a9,401.064797,8.151732e+10,2.544320e+06,1868
8,9,09,Bayern,Freistaat,13076721,2,60759,1599688800000,09,1ff920f4-62cd-4a4f-b8c9-f042f2a3e00a,464.634827,1.634855e+11,3.898618e+06,2644
9,10,10,Saarland,Land,990509,12,3218,1599688800000,10,e3396a6f-8a30-4fdf-8df7-def77dd38bea,324.883469,6.060692e+09,5.626789e+05,175


# API access vie REST service, e.g. USA data

example of a REST conform interface (Registration required)

www.smartable.ai

In [18]:
url_endpoint = 'https://api.smartable.ai/coronavirus/stats/US'

headers = {
# Request headers
'Cache-Control': 'no-cache',
'Subscription-Key': '7d7bd767d5fd4ff0b14f7ec9c26f8227',
}

response = requests.get(url_endpoint, headers = headers)

In [19]:
response

<Response [200]>

In [20]:
US_dict = json.loads(response.content)

with open('../data/raw/SMARTABLE/US_data.txt','w') as outfile:
    json.dump(US_dict, outfile, indent=2)